In [1]:
# Add the file to the staging area
!git add PredictNHL.ipynb

# Commit the file with a descriptive message
!git commit -m "Add PredictNHL.ipynb"

# Push the file to the main branch on GitHub
!git push origin main


[main 3bd0f7a] Add PredictNHL.ipynb

Auto packing the repository in background for optimum performance.
See "git help gc" for manual housekeeping.



 1 file changed, 759 insertions(+), 1109 deletions(-)


To https://github.com/nic01as1/NHLPredictor.git
   6896a62..3bd0f7a  main -> main


In [1]:
!pip install tensorflow
!pip install xgboost
!pip install streamlit
!pip install imbalanced-learn
!pip install shap
!pip install statsmodels



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, f1_score, classification_report, make_scorer
from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests


In [2]:
#AJOUTER LE NOMBRE DE JOURS DE REPOS ! et ajouter 2 autres mod`les (Neural et Logistic)

In [33]:
#Retrieve the rank of each teams

url = 'https://www.hockey-reference.com/leagues/NHL_2025.html#all_stats'

with webdriver.Chrome() as driver:
    driver.get(url)
    time.sleep(2) 
    soup = BeautifulSoup(driver.page_source, 'html.parser')

driver.quit()

table = soup.find('table', id='stats')

if table is None:
    print("Team Statistics table not found")
else:
    
    rows = table.find_all('tr')[1:] 
    teams = [
        row.find_all('td')[0].text.strip()
        for row in rows
        if row.find_all('td')
    ]
    ranks = list(range(1, len(teams) + 1)) 

    
    df_rank = pd.DataFrame({
        'Team': teams,
        'Rank': ranks,
        'Team Rank': [f"{team} {rank}" for team, rank in zip(teams, ranks)]
    })
    print(df_rank)

                     Team  Rank                 Team Rank
0     Washington Capitals     1     Washington Capitals 1
1           Winnipeg Jets     2           Winnipeg Jets 2
2    Vegas Golden Knights     3    Vegas Golden Knights 3
3          Minnesota Wild     4          Minnesota Wild 4
4            Dallas Stars     5            Dallas Stars 5
5       New Jersey Devils     6       New Jersey Devils 6
6         Edmonton Oilers     7         Edmonton Oilers 7
7     Toronto Maple Leafs     8     Toronto Maple Leafs 8
8     Carolina Hurricanes     9     Carolina Hurricanes 9
9      Colorado Avalanche    10     Colorado Avalanche 10
10       Florida Panthers    11       Florida Panthers 11
11      Los Angeles Kings    12      Los Angeles Kings 12
12          Boston Bruins    13          Boston Bruins 13
13    Tampa Bay Lightning    14    Tampa Bay Lightning 14
14         Calgary Flames    15         Calgary Flames 15
15  Columbus Blue Jackets    16  Columbus Blue Jackets 16
16      Vancou

In [35]:

#Retrieve the infos of EACH games for the season

def scrape_game_history(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    table = soup.find("table")
    if table is None:
        print(f"Table not found on {url}")
        return pd.DataFrame()  #

    
    data = {
        "Date": [],
        "Home Team": [],
        "Home Score": [],
        "Away Team": [],
        "Away Score": [],
        "Result": [],
        "Home Win": []
    }

    
    for row in table.find_all("tr")[1:]:  # Skip the header
        columns = row.find_all("td")
        if len(columns) < 6:  
            continue

        
        date = columns[1].text.strip()
        home_team = columns[3].text.strip()
        away_team = columns[4].text.strip()
        result = columns[5].text.strip()

        
        if "-" in result:  
            scores = result.split("-")
            if len(scores) == 2 and scores[0].strip().isdigit() and scores[1].strip().isdigit():
                home_score = int(scores[0].strip())
                away_score = int(scores[1].strip())
                home_win = home_score > away_score
            else:  
                home_score = away_score = home_win = None
        else:
            home_score = away_score = home_win = None

        
        data["Date"].append(date)
        data["Home Team"].append(home_team)
        data["Away Team"].append(away_team)
        data["Result"].append(result)
        data["Home Score"].append(home_score)
        data["Away Score"].append(away_score)
        data["Home Win"].append(home_win)

    
    df_game = pd.DataFrame(data)
    df_game["Date"] = df_game["Date"].apply(lambda x: x.split(" ")[0])  # Extract just the date
    return df_game



url_history = "https://fixturedownload.com/results/nhl-2024"


df_game = scrape_game_history(url_history)


if df_game.empty:
    print("No game data found.")
else:
    print(f"Scraped {len(df_game)} games.")
    print(df_game)

Scraped 1312 games.
            Date              Home Team  Home Score            Away Team  \
0     04/10/2024         Buffalo Sabres         1.0    New Jersey Devils   
1     05/10/2024      New Jersey Devils         3.0       Buffalo Sabres   
2     08/10/2024         Seattle Kraken         2.0      St. Louis Blues   
3     08/10/2024       Florida Panthers         6.0        Boston Bruins   
4     09/10/2024       Utah Hockey Club         5.0   Chicago Blackhawks   
...          ...                    ...         ...                  ...   
1307  17/04/2025    Toronto Maple Leafs         NaN    Detroit Red Wings   
1308  17/04/2025        Ottawa Senators         NaN  Carolina Hurricanes   
1309  17/04/2025       New York Rangers         NaN  Tampa Bay Lightning   
1310  17/04/2025    Pittsburgh Penguins         NaN  Washington Capitals   
1311  17/04/2025  Columbus Blue Jackets         NaN   New York Islanders   

      Away Score Result Home Win  
0            4.0  1 - 4    False

In [36]:
#Normalise Both datasource
team_name_mapping = {
    "Montréal Canadiens": "Montreal Canadiens", 
    
}


df_game["Home Team"] = df_game["Home Team"].replace(team_name_mapping)
df_game["Away Team"] = df_game["Away Team"].replace(team_name_mapping)


In [37]:
# Merge df_rank to add Home Team Rank
df_game = df_game.merge(
    df_rank.rename(columns={"Team": "Home Team", "Rank": "Home Team Rank"}), 
    on="Home Team", 
    how="left"
)

# Merge df_rank to add Away Team Rank
df_game = df_game.merge(
    df_rank.rename(columns={"Team": "Away Team", "Rank": "Away Team Rank"}), 
    on="Away Team", 
    how="left"
)

# Reorder the final columns
final_columns = [
    "Date", "Home Team", "Home Score", "Away Team", "Away Score",
    "Home Team Rank", "Away Team Rank", "Home Win"
]
df_final = df_game.loc[:, final_columns] 

print(df_final)


            Date              Home Team  Home Score            Away Team  \
0     04/10/2024         Buffalo Sabres         1.0    New Jersey Devils   
1     05/10/2024      New Jersey Devils         3.0       Buffalo Sabres   
2     08/10/2024         Seattle Kraken         2.0      St. Louis Blues   
3     08/10/2024       Florida Panthers         6.0        Boston Bruins   
4     09/10/2024       Utah Hockey Club         5.0   Chicago Blackhawks   
...          ...                    ...         ...                  ...   
1307  17/04/2025    Toronto Maple Leafs         NaN    Detroit Red Wings   
1308  17/04/2025        Ottawa Senators         NaN  Carolina Hurricanes   
1309  17/04/2025       New York Rangers         NaN  Tampa Bay Lightning   
1310  17/04/2025    Pittsburgh Penguins         NaN  Washington Capitals   
1311  17/04/2025  Columbus Blue Jackets         NaN   New York Islanders   

      Away Score  Home Team Rank  Away Team Rank Home Win  
0            4.0           

In [45]:
#DATA MANIPULATION


# Filter out the Unplayed Games
df_final = df_final.dropna(subset=["Home Score", "Away Score"])

# Convert Scores and Dates to Appropriate Data Types
df_final["Date"] = pd.to_datetime(df_final["Date"], format="%d/%m/%Y")
df_final[["Home Score", "Away Score"]] = df_final[["Home Score", "Away Score"]].astype(int)
df_final["Home Win"] = df_final["Home Win"].astype(bool)

# Sort Data by Date for Chronological Calculations
df_final = df_final.sort_values(by="Date").reset_index(drop=True)

# Function: Calculate Last 10 Games Stats
def calculate_last_10_stats(df, team_column):
    last_10_wins = []
    team_games = {team: [] for team in pd.concat([df["Home Team"], df["Away Team"]]).unique()}

    for _, row in df.iterrows():
        team = row[team_column]
        recent_games = team_games[team][-10:]  # Last 10 games
        last_10_wins.append(sum(recent_games))
        team_games[row["Home Team"]].append(row["Home Win"])
        team_games[row["Away Team"]].append(not row["Home Win"])
    return last_10_wins

# Add Last 10 Wins for Home and Away Teams
df_final["Home Last 10 Wins"] = calculate_last_10_stats(df_final, "Home Team")
df_final["Away Last 10 Wins"] = calculate_last_10_stats(df_final, "Away Team")

# Add Whether Teams Played Yesterday
def calculate_played_yesterday(df, team_column):
    played_yesterday = []
    last_game_date = {}

    for _, row in df.iterrows():
        team = row[team_column]
        played_yesterday.append(last_game_date.get(team) == row["Date"] - pd.Timedelta(days=1))
        last_game_date[team] = row["Date"]
    return played_yesterday

df_final["Home Played Yesterday"] = calculate_played_yesterday(df_final, "Home Team")
df_final["Away Played Yesterday"] = calculate_played_yesterday(df_final, "Away Team")

# Add Win Rate for Home and Away Teams
def calculate_win_rate(df, team_column, is_home_column):
    win_rate = []
    team_stats = {team: {"wins": 0, "games": 0} for team in pd.concat([df["Home Team"], df["Away Team"]]).unique()}

    for _, row in df.iterrows():
        team = row[team_column]
        stats = team_stats[team]
        win_rate.append(stats["wins"] / stats["games"] if stats["games"] > 0 else 0)
        
        # Update stats after calculating win rate
        if is_home_column:
            stats["wins"] += row["Home Win"]
        else:
            stats["wins"] += not row["Home Win"]
        stats["games"] += 1
    return win_rate

df_final["Home Win Rate"] = calculate_win_rate(df_final, "Home Team", is_home_column=True)
df_final["Away Win Rate"] = calculate_win_rate(df_final, "Away Team", is_home_column=False)

# Add Overall Win Streak for Home and Away Teams
def calculate_overall_win_streak(df):
    streak = {}
    home_streaks = []
    away_streaks = []

    for _, row in df.iterrows():
        home_team = row["Home Team"]
        away_team = row["Away Team"]

        
        home_streaks.append(streak.get(home_team, 0))
        away_streaks.append(streak.get(away_team, 0))

        
        if row["Home Win"]:
            streak[home_team] = streak.get(home_team, 0) + 1
            streak[away_team] = 0
        else:
            streak[home_team] = 0
            streak[away_team] = streak.get(away_team, 0) + 1
    return home_streaks, away_streaks

home_streaks, away_streaks = calculate_overall_win_streak(df_final)
df_final["Home Team Overall Win Streak Before Game"] = home_streaks
df_final["Away Team Overall Win Streak Before Game"] = away_streaks

# Add Average Opponent Strength for Home and Away Teams
def calculate_avg_opponent_rank(df, team_column, opponent_rank_column):
    avg_opponent_rank = []
    opponent_stats = {team: [] for team in pd.concat([df["Home Team"], df["Away Team"]]).unique()}

    for _, row in df.iterrows():
        team = row[team_column]
        avg_opponent_rank.append(
            np.mean(opponent_stats[team][-10:]) if opponent_stats[team] else np.nan
        )
        opponent_stats[row["Home Team"]].append(row["Away Team Rank"])
        opponent_stats[row["Away Team"]].append(row["Home Team Rank"])
    return avg_opponent_rank

df_final["Home Opponent Strength"] = calculate_avg_opponent_rank(df_final, "Home Team", "Away Team Rank")
df_final["Away Opponent Strength"] = calculate_avg_opponent_rank(df_final, "Away Team", "Home Team Rank")

def calculate_days_since_last_game(df):
    last_game_date = {}  # Dictionary to track the last game date for each team
    home_days_since_last_game = []
    away_days_since_last_game = []

    for _, row in df.iterrows():
        home_team = row["Home Team"]
        away_team = row["Away Team"]
        current_date = row["Date"]

        # Calculate days since last game for the home team
        if home_team in last_game_date:
            days_since_last_home_game = (current_date - last_game_date[home_team]).days
        else:
            days_since_last_home_game = None  # No previous game played
        home_days_since_last_game.append(days_since_last_home_game)

        # Calculate days since last game for the away team
        if away_team in last_game_date:
            days_since_last_away_game = (current_date - last_game_date[away_team]).days
        else:
            days_since_last_away_game = None  # No previous game played
        away_days_since_last_game.append(days_since_last_away_game)

        # Update the last game date for both teams
        last_game_date[home_team] = current_date
        last_game_date[away_team] = current_date

    return home_days_since_last_game, away_days_since_last_game


# Apply the function to the DataFrame
home_days, away_days = calculate_days_since_last_game(df_final)

# Add the calculated values as new columns
df_final["Home Team Days Since Last Game"] = home_days
df_final["Away Team Days Since Last Game"] = away_days

print(df_final[["Date", "Home Team", "Away Team", "Home Opponent Strength", "Away Opponent Strength","Home Last 10 Wins","Away Last 10 Wins","Home Played Yesterday","Away Played Yesterday","Home Win Rate","Away Win Rate","Home Team Overall Win Streak Before Game","Away Team Overall Win Streak Before Game","Away Team Days Since Last Game","Home Team Days Since Last Game"]].tail(10))

# Save the Enhanced Dataset to CSV
df_final.to_csv("enhanced_game_data.csv", index=False)


          Date              Home Team             Away Team  \
693 2025-01-15    Toronto Maple Leafs          Dallas Stars   
694 2025-01-15    Washington Capitals         Anaheim Ducks   
695 2025-01-15  Columbus Blue Jackets   Philadelphia Flyers   
696 2025-01-15      New Jersey Devils      Florida Panthers   
697 2025-01-15    Nashville Predators  Vegas Golden Knights   
698 2025-01-15          Winnipeg Jets     Vancouver Canucks   
699 2025-01-15     New York Islanders       Ottawa Senators   
700 2025-01-15        St. Louis Blues        Calgary Flames   
701 2025-01-15     Colorado Avalanche      New York Rangers   
702 2025-01-15       Utah Hockey Club    Montreal Canadiens   

     Home Opponent Strength  Away Opponent Strength  Home Last 10 Wins  \
693                    16.3                    21.6                  6   
694                    17.8                    12.4                  5   
695                    18.7                    15.3                  7   
696       

In [47]:

results = df_final
results.head()

,Date,Home Team,Home Score,Away Team,Away Score,Home Team Rank,Away Team Rank,Home Win,Home Last 10 Wins,Away Last 10 Wins,Home Played Yesterday,Away Played Yesterday,Home Win Rate,Away Win Rate,Home Team Overall Win Streak Before Game,Away Team Overall Win Streak Before Game,Home Opponent Strength,Away Opponent Strength,Home Team Days Since Last Game,Away Team Days Since Last Game
0,2024-10-04,Buffalo Sabres,1,New Jersey Devils,4,29,6,False,0,0,False,False,0.0,0.0,0,0,NaN,NaN,NaN,NaN
1,2024-10-05,New Jersey Devils,3,Buffalo Sabres,1,6,29,True,1,0,False,False,0.0,0.0,1,0,29.0,6.0,1.0,1.0
2,2024-10-08,Seattle Kraken,2,St. Louis Blues,3,27,20,False,0,0,False,False,0.0,0.0,0,0,NaN,NaN,NaN,NaN
3,2024-10-08,Florida Panthers,6,Boston Bruins,4,11,13,True,0,0,False,False,0.0,0.0,0,0,NaN,NaN,NaN,NaN
4,2024-10-09,Utah Hockey Club,5,Chicago Blackhawks,2,25,32,True,0,0,False,False,0.0,0.0,0,0,NaN,NaN,NaN,NaN


In [48]:

scorer = make_scorer(f1_score, pos_label=None, average='weighted')

# Define y_true based on the Home Win column
y_true = df_final["Home Win"].astype(int)  # Convert True/False to 1/0

# Create a dummy prediction where all predictions are 1 (Home Win = True)
y_pred = [1] * len(y_true)

# Calculate the weighted F1 score
f1 = f1_score(y_true, y_pred, pos_label=None, average="weighted")

# Print the F1 score
print(f"F1 Score: {f1:.4f}")

F1 Score: 0.3793


In [50]:

# Load your dataset (assuming it's stored in `df_final`)
# Ensure the target variable is properly defined
target = "Home Win" 
features = [
     "Home Opponent Strength", 
    "Away Opponent Strength",
    "Home Last 10 Wins",
    "Away Last 10 Wins",
    "Home Played Yesterday",
    "Away Played Yesterday",
    "Home Win Rate",
    "Away Win Rate",
    "Home Team Overall Win Streak Before Game",
    "Away Team Overall Win Streak Before Game",
    "Away Team Days Since Last Game",
    "Home Team Days Since Last Game"
    
]  # Select only numerical and categorical features for the model

# Prepare the dataset
X = df_final[features]
y = df_final[target].astype(int)  # Convert True/False to 1/0

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred, average="weighted")
print(f"F1 Score of the model: {f1:.4f}")

# Get feature importances from the Random Forest
importances = model.feature_importances_
feature_importance_df = pd.DataFrame({"Feature": features, "Importance": importances})
feature_importance_df = feature_importance_df.sort_values(by="Importance", ascending=False)
print("Feature Importance from Random Forest:")
print(feature_importance_df)

# Calculate permutation importance
perm_importance = permutation_importance(model, X_test, y_test, scoring="f1_weighted", random_state=42)
perm_importance_df = pd.DataFrame({
    "Feature": features,
    "Importance": perm_importance.importances_mean
}).sort_values(by="Importance", ascending=False)
print("\nPermutation Importance:")
print(perm_importance_df)
#"Away Opponent Strength"
# Save feature importances to CSV files
feature_importance_df.to_csv("feature_importance_random_forest.csv", index=False)
perm_importance_df.to_csv("feature_importance_permutation.csv", index=False)

F1 Score of the model: 0.5378
Feature Importance from Random Forest:
                                     Feature  Importance
0                     Home Opponent Strength    0.158553
1                     Away Opponent Strength    0.153517
6                              Home Win Rate    0.134833
7                              Away Win Rate    0.134764
3                          Away Last 10 Wins    0.080442
2                          Home Last 10 Wins    0.080156
10            Away Team Days Since Last Game    0.066827
11            Home Team Days Since Last Game    0.056636
9   Away Team Overall Win Streak Before Game    0.056233
8   Home Team Overall Win Streak Before Game    0.051708
5                      Away Played Yesterday    0.014342
4                      Home Played Yesterday    0.011989

Permutation Importance:
                                     Feature  Importance
2                          Home Last 10 Wins    0.060964
7                              Away Win Rate    0.0

In [17]:
# Retain only the important features with positive permutation importance
selected_features = [
    "Home Team Rank",               # High importance in both analyses
    "Away Team Rank",               # High importance in both analyses
    "Home Opponent Strength",       # High in Random Forest
    "Away Opponent Strength",       # High in Random Forest
    "Home Win Rate",                # High in both analyses
    "Away Win Rate",                # High in both analyses
    "Home Played Yesterday",        # Appears in both
    "Away Played Yesterday",        # Appears in both
    "Home Last 10 Wins",            # Included for diversity despite low score
    "Away Last 10 Wins",            # Included for diversity despite low score
    "Home Team Overall Win Streak Before Game",  # Decent in permutation importance
    "Away Team Overall Win Streak Before Game"   # Decent in permutation importance
]

X_selected = df_final[selected_features]
y = df_final["Home Win"].astype(int)

# Train-test split
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Train the Random Forest model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred, average="weighted")
print(f"F1 Score with selected features: {f1:.4f}")


F1 Score with selected features: 0.5698


In [18]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
encoding = LabelEncoder() # encoding object has LabelEncoder ready to use
encoding.fit(results["Home Team"].values) # give each team a number to identify them by

LabelEncoder()

In [19]:
home_teams = encoding.transform(results["Home Team"].values)
visitor_teams = encoding.transform(results["Away Team"].values)

# vstack allows you to stack a sequence of arrays vertically - in this case we do it to make easy comparisons
X_teams = np.vstack([home_teams, visitor_teams]).T

X_teams[:5], X_teams.shape

(array([[ 2, 16],
        [16,  2],
        [23, 24],
        [11,  1],
        [27,  5]]),
 (703, 2))

In [20]:
# Prepare the One Hot Encoder
onehot = OneHotEncoder()
X_teams = onehot.fit_transform(X_teams).toarray()


In [21]:

X_teams.shape # see matrix dimensions

(703, 64)

In [22]:
# Lets build our model using the Decision Tree Classifier
tree_clf = DecisionTreeClassifier(random_state=10)
scores = cross_val_score(tree_clf, X_teams, y_true, scoring=scorer)

# Note: The only feature is the encoder we just made
print("The F1 score using encoded teams is the following: {0:.4f}".format(np.mean(scores)))

The F1 score using encoded teams is the following: 0.5521


In [23]:

forest_clf = RandomForestClassifier(random_state=10)
scores = cross_val_score(forest_clf, X_teams, y_true, scoring=scorer)
print("The F1 score using Random Forest Classifier is the following: {0:.4f}".format(np.mean(scores)))

The F1 score using Random Forest Classifier is the following: 0.5685


In [24]:
# Define the parameter grid with valid values for max_features
parameter_space = {
    "max_features": [2, 10, 50, 'sqrt', 'log2', None],  # Replace 'auto' with valid options
    "n_estimators": [50, 100, 200],
    "criterion": ["gini", "entropy"],
    "min_samples_leaf": [1, 2, 4, 6],
}

# Initialize the Random Forest model and the scoring function
forest_clf = RandomForestClassifier(random_state=10)
scorer = make_scorer(f1_score, pos_label=None, average='weighted')

# Set up GridSearchCV
grid = GridSearchCV(forest_clf, parameter_space, scoring=scorer, error_score='raise')

# Fit the model
try:
    grid.fit(X_teams, y_true)
except Exception as e:
    print(f"An error occurred during grid search: {e}")

# View results if the fit is successful
print("F1: {0:.4f}".format(grid.best_score_))
print("Best Parameters:", grid.best_params_)
print("Best Estimator:", grid.best_estimator_)

F1: 0.5994
Best Parameters: {'criterion': 'entropy', 'max_features': 'log2', 'min_samples_leaf': 4, 'n_estimators': 50}
Best Estimator: RandomForestClassifier(criterion='entropy', max_features='log2',
                       min_samples_leaf=4, n_estimators=50, random_state=10)


In [25]:
# Extract the features
X_homehigher = results[selected_features].values

# Combine with additional features (e.g., one-hot encoded team features)
X_all = np.hstack([X_homehigher, X_teams])
print("Feature matrix shape:", X_all.shape)

# Define the scorer (e.g., F1 score)
scorer = make_scorer(f1_score, average="weighted")

# Decision Tree Classifier
tree_clf = DecisionTreeClassifier(random_state=10)
tree_scores = cross_val_score(tree_clf, X_all, y_true, scoring=scorer, cv=5)
print("F1 score for Decision Tree: {0:.4f}".format(np.mean(tree_scores)))

# Random Forest Classifier
rf_clf = RandomForestClassifier(random_state=10, n_estimators=100, max_depth=None)
rf_scores = cross_val_score(rf_clf, X_all, y_true, scoring=scorer, cv=5)
print("F1 score for Random Forest: {0:.4f}".format(np.mean(rf_scores)))

Feature matrix shape: (703, 76)
F1 score for Decision Tree: 0.5049
F1 score for Random Forest: 0.5887


In [26]:
# Define the parameter grid with valid values for max_features
parameter_space = {
    "max_features": [2, 10, 50, 'sqrt', 'log2', None],  # Replaced 'auto' with valid options
    "n_estimators": [50, 100, 200],
    "criterion": ["gini", "entropy"],
    "min_samples_leaf": [1, 2, 4, 6],
}

# Initialize the Random Forest model and scoring function
forest_clf = RandomForestClassifier(random_state=10)
scorer = make_scorer(f1_score, pos_label=None, average='weighted')

# Set up GridSearchCV
grid = GridSearchCV(forest_clf, parameter_space, scoring=scorer, error_score='raise')  # Added error_score='raise'

# Fit the model
try:
    grid.fit(X_all, y_true)
except Exception as e:
    print(f"An error occurred during grid search: {e}")

# Display results if the fit is successful
print("F1 score: {0:.4f}".format(grid.best_score_))
print("Best Parameters:", grid.best_params_)
print("Best Estimator:", grid.best_estimator_)

F1 score: 0.6193
Best Parameters: {'criterion': 'entropy', 'max_features': 'log2', 'min_samples_leaf': 2, 'n_estimators': 200}
Best Estimator: RandomForestClassifier(criterion='entropy', max_features='log2',
                       min_samples_leaf=2, n_estimators=200, random_state=10)


In [28]:

selected_features = [
    "Home Team Rank",               # High importance in both analyses
    "Away Team Rank",               # High importance in both analyses
    "Home Opponent Strength",       # High in Random Forest
    "Away Opponent Strength",       # High in Random Forest
    "Home Win Rate",                # High in both analyses
    "Away Win Rate",                # High in both analyses
    "Home Played Yesterday",        # Appears in both
    "Away Played Yesterday",        # Appears in both
    "Home Last 10 Wins",            # Included for diversity despite low score
    "Away Last 10 Wins",            # Included for diversity despite low score
    "Home Team Overall Win Streak Before Game",  # Decent in permutation importance
    "Away Team Overall Win Streak Before Game"   # Decent in permutation importance
]

# List of all teams (from encoding.classes_)
teams = list(encoding.classes_)

# Display the list of teams
print("Available Teams:")
for i, team in enumerate(teams):
    print(f"{i + 1}: {team}")

# Let the user select home and away teams by number
home_team_index = int(input("Select the home team by number: ")) - 1
away_team_index = int(input("Select the away team by number: ")) - 1

# Get the team names based on user selection
home_team = teams[home_team_index]
away_team = teams[away_team_index]

print(f"Home Team: {home_team}")
print(f"Away Team: {away_team}")

# Retrieve the selected features for the teams
input_data = results[
    (results["Home Team"] == home_team) & (results["Away Team"] == away_team)
][selected_features]

if input_data.empty:
    print("No matching game found for the selected teams in the dataset.")
else:
    # Create one-hot encoding for the teams
    one_hot_home = [1 if team == home_team else 0 for team in teams]
    one_hot_away = [1 if team == away_team else 0 for team in teams]

    # Combine features into a single input vector
    combined_features = np.hstack([
        input_data.iloc[0].values,  # Selected features
        one_hot_home,  # One-hot encoded home team
        one_hot_away   # One-hot encoded away team
    ])

    # Ensure the input vector matches the model's expected format
    input_vector = combined_features.reshape(1, -1)  # Reshape for prediction

    # Make the prediction using the best model
    predicted_prob = grid.best_estimator_.predict_proba(input_vector)

    # Extract probabilities for home and away teams
    home_team_prob = predicted_prob[0][1]  # Probability of home team winning
    away_team_prob = predicted_prob[0][0]  # Probability of away team winning

    # Input: Gambling site odds
    home_team_odds = float(input(f"Enter the odds for {home_team}: "))
    away_team_odds = float(input(f"Enter the odds for {away_team}: "))

    # Kelly Criterion Formula
    def kelly_criterion(prob, odds):
        b = odds - 1
        q = 1 - prob
        return max(0, (b * prob - q) / b)  # Return 0 if Kelly fraction is negative

    # Calculate Kelly Criterion for both teams
    home_kelly = kelly_criterion(home_team_prob, home_team_odds)
    away_kelly = kelly_criterion(away_team_prob, away_team_odds)

    # Determine the best bet
    if home_kelly > away_kelly:
        print(f"Bet on {home_team}. Kelly Criterion suggests betting {home_kelly:.2%} of your bankroll.")
    elif away_kelly > home_kelly:
        print(f"Bet on {away_team}. Kelly Criterion suggests betting {away_kelly:.2%} of your bankroll.")
    else:
        print("Neither bet has a positive expected value according to the Kelly Criterion.")

    # Output additional details
    print(f"Home Team: {home_team}, Probability: {home_team_prob:.2%}, Odds: {home_team_odds}")
    print(f"Away Team: {away_team}, Probability: {away_team_prob:.2%}, Odds: {away_team_odds}")
    print(f"Kelly Criterion (Home): {home_kelly:.2%}")
    print(f"Kelly Criterion (Away): {away_kelly:.2%}")
    

Available Teams:
1: Anaheim Ducks
2: Boston Bruins
3: Buffalo Sabres
4: Calgary Flames
5: Carolina Hurricanes
6: Chicago Blackhawks
7: Colorado Avalanche
8: Columbus Blue Jackets
9: Dallas Stars
10: Detroit Red Wings
11: Edmonton Oilers
12: Florida Panthers
13: Los Angeles Kings
14: Minnesota Wild
15: Montreal Canadiens
16: Nashville Predators
17: New Jersey Devils
18: New York Islanders
19: New York Rangers
20: Ottawa Senators
21: Philadelphia Flyers
22: Pittsburgh Penguins
23: San Jose Sharks
24: Seattle Kraken
25: St. Louis Blues
26: Tampa Bay Lightning
27: Toronto Maple Leafs
28: Utah Hockey Club
29: Vancouver Canucks
30: Vegas Golden Knights
31: Washington Capitals
32: Winnipeg Jets


Select the home team by number:  14
Select the away team by number:  11


Home Team: Minnesota Wild
Away Team: Edmonton Oilers


Enter the odds for Minnesota Wild:  2.85
Enter the odds for Edmonton Oilers:  1.44


Neither bet has a positive expected value according to the Kelly Criterion.
Home Team: Minnesota Wild, Probability: 33.47%, Odds: 2.85
Away Team: Edmonton Oilers, Probability: 66.53%, Odds: 1.44
Kelly Criterion (Home): 0.00%
Kelly Criterion (Away): 0.00%
